## cropped images - axial T2

# Stage 1

In [2]:
%%capture
!pip install '/kaggle/input/rsna2024-demo-workflow/natsort-8.4.0-py3-none-any.whl'
!pip install /kaggle/input/notebook0248a0e3f9/pycocotools-2.0.7-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl /kaggle/input/notebook0248a0e3f9/dicomsdl-0.109.3-cp310-cp310-manylinux_2_12_x86_64.manylinux2010_x86_64.whl /kaggle/input/notebook0248a0e3f9/loguru-0.7.2-py3-none-any.whl /kaggle/input/notebook0248a0e3f9/pydicom-2.4.4-py3-none-any.whl /kaggle/input/notebook0248a0e3f9/python_gdcm-3.0.24.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl  /kaggle/input/notebook97381653b4/ensemble_boxes-1.0.9-py3-none-any.whl

!pip install -qq /kaggle/working/python-packages/timm-0.9.10-py3-none-any.whl

# 1, axial level estimation by hengck23's code
ref: https://www.kaggle.com/code/hengck23/ver-1-demo-workflow-2-stage-approach?scriptVersionId=191553260

In [7]:
from glob import glob
debug = False
debug_fold = 2
plot = False
train_test = 'train'

import sys, os
sys.path.append('/kaggle/input/rsna2024-demo-workflow')

from _dir_setting_ import *

import matplotlib
import matplotlib.pyplot as plt

from helper import *
from data import *
from model import *

# STEP0 : SETUP DATA ==========================

cfg=dotdict(
    point_net=dotdict(
        checkpoint='/kaggle/input/rsna2024-demo-workflow/00002484.pth',
        image_size=160,
    ),
)

level_to_label={
    'l1_l2':1,
    'l2_l3':2,
    'l3_l4':3,
    'l4_l5':4,
    'l5_s1':5,
}

#################################################

# study id used for demo
id_df = pd.read_csv(f'{DATA_KAGGLE_DIR}/test_series_descriptions.csv')    

point_net = Net(pretrained=False)
f = torch.load(cfg.point_net.checkpoint, map_location=lambda storage, loc: storage)
state_dict = f['state_dict']
point_net.load_state_dict(state_dict, strict=False)
point_net.cuda()
point_net.eval()
point_net.output_type = ['infer']


from _dir_setting_ import *
from natsort import natsorted
import pandas as pd
pd.set_option('mode.chained_assignment', None) # disable SettingWithCopyWarning

import numpy as np
import pydicom
import glob
import timm
import cv2

from matplotlib.patches import FancyArrowPatch
from mpl_toolkits.mplot3d import proj3d
from pdb import set_trace as st

class dotdict(dict):
    __setattr__ = dict.__setitem__
    __delattr__ = dict.__delitem__

    def __getattr__(self, name):
        try:
            return self[name]
        except KeyError:
            raise AttributeError(name)

################################################################################3


# read into volume (np_array) + dicom_header(df)
'''
import notes:
- instance_number may not be sequentially (can have missing num)
- instance_number is 1-indexed

'''

## 3d/2d processing #########################################################
def np_dot(a,b):
    return np.sum(a * b, 1)

def project_to_3d(x,y,z, df):
    d = df.iloc[z]
    H, W = d.H, d.W
    sx, sy, sz = [float(v) for v in d.ImagePositionPatient]
    o0, o1, o2, o3, o4, o5, = [float(v) for v in d.ImageOrientationPatient]
    delx, dely = d.PixelSpacing

    xx = o0 * delx * x + o3 * dely * y + sx
    yy = o1 * delx * x + o4 * dely * y + sy
    zz = o2 * delx * x + o5 * dely * y + sz
    return xx,yy,zz

## read data #########################################################
def resize_volume(volume, image_size):
    image = volume.copy()
    image = np.ascontiguousarray(image.transpose((1, 2, 0)))
    image = cv2.resize(image, (image_size, image_size), interpolation=cv2.INTER_LINEAR)
    image = np.ascontiguousarray(image.transpose((2, 0, 1)))  # cv2.INTER_LINEAR=1
    return image

def normalise_to_8bit(x, lower=0.1, upper=99.9): # 1, 99 #0.05, 99.5 #0, 100
    lower, upper = np.percentile(x, (lower, upper))
    x = np.clip(x, lower, upper)
    x = x - np.min(x)
    x = x / np.max(x)
    return (x * 255).astype(np.uint8)
def read_series(study_id,series_id,series_description):
    data_kaggle_dir = DATA_KAGGLE_DIR
    dicom_dir = f'{data_kaggle_dir}/{train_test}_images/{study_id}/{series_id}'

    # read dicom file
    dicom_file = natsorted(glob.glob(f'{dicom_dir}/*.dcm'))
    instance_number = [int(f.split('/')[-1].split('.')[0]) for f in dicom_file]
    dicom = [pydicom.dcmread(f) for f in dicom_file]

    # make dicom header df
    H, W = dicom[0].pixel_array.shape
    dicom_df = []
    for i, d in zip(instance_number, dicom):  # d__.dict__
        dicom_df.append(
            dotdict(
                study_id=study_id,
                series_id=series_id,
                series_description=series_description,
                instance_number=i,
                # InstanceNumber = d.InstanceNumber,
                ImagePositionPatient=[float(v) for v in d.ImagePositionPatient],
                ImageOrientationPatient=[float(v) for v in d.ImageOrientationPatient],
                PixelSpacing=[float(v) for v in d.PixelSpacing],
                SpacingBetweenSlices=float(d.SpacingBetweenSlices),
                SliceThickness=float(d.SliceThickness),
                grouping=str([round(float(v), 3) for v in d.ImageOrientationPatient]),
                H=H,
                W=W,
            )
        )
    dicom_df = pd.DataFrame(dicom_df)

    # sort slices
    dicom_df = [d for _, d in dicom_df.groupby('grouping')]

    data = []
    sort_data_by_group = []
    for df in dicom_df:
        position = np.array(df['ImagePositionPatient'].values.tolist())
        orientation = np.array(df['ImageOrientationPatient'].values.tolist())
        normal = np.cross(orientation[:, :3], orientation[:, 3:])
        projection = np_dot(normal, position)
        df.loc[:, 'projection'] = projection
        df = df.sort_values('projection')


        # todo: assert all slices are continous ??
        # use  (position[-1]-position[0])/N = SpacingBetweenSlices ??
        assert len(df.SliceThickness.unique()) == 1
        assert len(df.SpacingBetweenSlices.unique()) == 1


        volume = [
            dicom[instance_number.index(i)].pixel_array for i in df.instance_number
        ]
        volume = np.stack(volume)
        volume = normalise_to_8bit(volume)
        data.append(dotdict(
            df=df,
            volume=volume,
        ))

        if 'sagittal' in series_description.lower():
            sort_data_by_group.append(position[0, 0])  # x
        if 'axial' in series_description.lower():
            sort_data_by_group.append(position[0, 2])  # z

    data = [r for _, r in sorted(zip(sort_data_by_group, data))]
    for i, r in enumerate(data):
        r.df.loc[:, 'group'] = i

    df = pd.concat([r.df for r in data])
    df.loc[:, 'z'] = np.arange(len(df))
    try:    
        volume = np.concatenate([r.volume for r in data])
    except:
        volume = []
        for r in data:
            # st()
            volume.append([cv2.resize(im, (608, 608)) for im in r.volume])
        volume = np.concatenate(volume)
    data = dotdict(
        series_id=series_id,
        df=df,
        volume=volume,
    )
    return data
def read_axial_df(study_id,series_id,series_description):
    data_kaggle_dir = DATA_KAGGLE_DIR
    dicom_dir = f'{data_kaggle_dir}/{train_test}_images/{study_id}/{series_id}'

    # read dicom file
    dicom_file = natsorted(glob.glob(f'{dicom_dir}/*.dcm'))
    instance_number = [int(f.split('/')[-1].split('.')[0]) for f in dicom_file]
    dicom = [pydicom.dcmread(f) for f in dicom_file]

    # make dicom header df
    H, W = dicom[0].pixel_array.shape
    dicom_df = []
    for i, d in zip(instance_number, dicom):  # d__.dict__
        dicom_df.append(
            dotdict(
                study_id=study_id,
                series_id=series_id,
                series_description=series_description,
                instance_number=i,
                # InstanceNumber = d.InstanceNumber,
                ImagePositionPatient=[float(v) for v in d.ImagePositionPatient],
                ImageOrientationPatient=[float(v) for v in d.ImageOrientationPatient],
                PixelSpacing=[float(v) for v in d.PixelSpacing],
                SpacingBetweenSlices=float(d.SpacingBetweenSlices),
                SliceThickness=float(d.SliceThickness),
                grouping=str([round(float(v), 3) for v in d.ImageOrientationPatient]),
                H=H,
                W=W,
            )
        )
    dicom_df = pd.DataFrame(dicom_df)

    # sort slices
    dicom_df = [d for _, d in dicom_df.groupby('grouping')]

    data = []
    sort_data_by_group = []
    for df in dicom_df:
        position = np.array(df['ImagePositionPatient'].values.tolist())
        orientation = np.array(df['ImageOrientationPatient'].values.tolist())
        normal = np.cross(orientation[:, :3], orientation[:, 3:])
        projection = np_dot(normal, position)
        df.loc[:, 'projection'] = projection
        df = df.sort_values('projection')


        # todo: assert all slices are continous ??
        # use  (position[-1]-position[0])/N = SpacingBetweenSlices ??
        assert len(df.SliceThickness.unique()) == 1
        if len(df.SpacingBetweenSlices.unique()) != 1:
            import pdb;pdb.set_trace()
        assert len(df.SpacingBetweenSlices.unique()) == 1


        volume = [
            dicom[instance_number.index(i)].pixel_array for i in df.instance_number
        ]
        volume = np.stack(volume)
        volume = normalise_to_8bit(volume)
        data.append(dotdict(
            df=df,
            volume=volume,
        ))

        if 'sagittal' in series_description.lower():
            sort_data_by_group.append(position[0, 0])  # x
        if 'axial' in series_description.lower():
            sort_data_by_group.append(position[0, 2])  # z

    data = [r for _, r in sorted(zip(sort_data_by_group, data))]
    for i, r in enumerate(data):
        r.df.loc[:, 'group'] = i

    df = pd.concat([r.df for r in data])
    df.loc[:, 'z'] = np.arange(len(df))
    return df

def read_study(study_id, sagittal_t2_id, axial_t2_id):
    return dotdict(
        study_id = study_id,
        sagittal_t2 =read_series(study_id, sagittal_t2_id, 'sagittal_t2'),
        axial_t2 =read_series(study_id, axial_t2_id, 'axial_t2'),
    )
def get_true_sagittal_t2_point(study_id, sagittal_t2_df):
    series_id = sagittal_t2_df.iloc[0].series_id
    label_coord_df = pd.read_csv(f'{DATA_KAGGLE_DIR}/{train_test}_label_coordinates.csv')
    label_df = label_coord_df[
         (label_coord_df.study_id == study_id)
       & (label_coord_df.series_id == series_id)
    ]
    label_df = label_df.sort_values('level')
    point=label_df[['x','y']].values
    instance_number = label_df.instance_number.values

    #mapping from instance num to z (array index)
    map_instance_number, map_z = sagittal_t2_df[['instance_number','z',]].values.T
    map = {n:z for n,z in zip(map_instance_number,map_z)}
    z = [map[n] for n in instance_number]
    return point,z

############################################################3
#post processing (sagittal_t2 point net)
def probability_to_point(probability, threshold=0.5):
    #todo: handle mssing point
    point=[]
    for l in range(1, 6):
        y, x = np.where(probability[l] > threshold)
        y = round(y.mean())
        x = round(x.mean())
        point.append((x, y))
    return point


def view_to_world(sagittal_t2_point, z, sagittal_t2_df, image_size):

    H = sagittal_t2_df.iloc[0].H
    W = sagittal_t2_df.iloc[0].W
    scale_x = W / image_size
    scale_y = H / image_size

    xxyyzz = []
    for l in range(1, 6):
        x,y = sagittal_t2_point[l-1]
        xx,yy,zz = project_to_3d(x*scale_x, y*scale_y, z, sagittal_t2_df)
        xxyyzz.append((xx, yy, zz))

    xxyyzz = np.array(xxyyzz)
    return xxyyzz

def point_to_level(world_point, axial_t2_df):

    # we get closest axial slices (z) to the CSC world points

    xxyyzz = world_point
    orientation = np.array(axial_t2_df.ImageOrientationPatient.values.tolist())
    position = np.array(axial_t2_df.ImagePositionPatient.values.tolist())
    ox = orientation[:, :3]
    oy = orientation[:, 3:]
    oz = np.cross(ox, oy)
    t = xxyyzz.reshape(-1, 1, 3) - position.reshape(1, -1, 3)
    dis = (oz.reshape(1, -1, 3) * t).sum(-1)  # np.dot(point-s,oz)
    fdis = np.fabs(dis)
    closest_z = fdis.argmin(-1)
    closest_fdis = fdis.min(-1)
    closest_df = axial_t2_df.iloc[closest_z]

    if 1:
        #<todo> hard/soft assigment, multi/single assigment
        # no assignment based on distance

        # allow point found in multi group
        num_group   = len(axial_t2_df['group'].unique())
        point_group = axial_t2_df.group.values[fdis.argsort(-1)[:, :3]].tolist()
        point_group = [list(set(g)) for g in point_group]
        group_point = [[] for g in range(num_group)]
        for l in range(5):
            for k in point_group[l]:
                group_point[k].append(l)
                # print(k)
                # print(group_point[k])
                # print(group_point)
        group_point = [sorted(list(set(g))) for g in group_point]

    D = len(axial_t2_df)
    assigned_level=np.full(D,fill_value=0, dtype=int)
    for group in range(num_group):
        point_in_this_group = np.array(group_point[group])  # np.where(closest_df['group'] == group)[0]
        slice_in_this_group = np.where(axial_t2_df['group'] == group)[0]
        if len(point_in_this_group) == 0:
            continue # unassigned, level=0

        level = point_in_this_group[fdis[point_in_this_group][:, slice_in_this_group].argmin(0)] + 1
        assigned_level[slice_in_this_group] = level

    # sor =  (fdis.argmin(0)+1)[closest_z]
    # closest_z= [ fdis.argmin(0)+1]
    return assigned_level, closest_z, closest_fdis #dis is soft assignment


#########################################################################3
#visualisation
level_color = [
    [0, 0, 0],
    [255, 0, 0],
    [0, 255, 0],
    [0, 0, 255],
    [255, 255, 0],
    [0, 255, 255],
]

def probability_to_rgb(probability):
    _6_,H,W = probability.shape
    rgb = np.zeros((H, W, 3))
    for i in range(1, 6):
        rgb += probability[i].reshape(H, W, 1) * [[level_color[i]]]
    rgb = rgb.astype(np.uint8)
    return rgb


class Arrow3D(FancyArrowPatch):
    def __init__(self, xs, ys, zs, *args, **kwargs):
        super().__init__((0,0), (0,0), *args, **kwargs)
        self._verts3d = xs, ys, zs

    def do_3d_projection(self, renderer=None):
        xs3d, ys3d, zs3d = self._verts3d
        xs, ys, zs = proj3d.proj_transform(xs3d, ys3d, zs3d, self.axes.M)
        self.set_positions((xs[0],ys[0]),(xs[1],ys[1]))
        return np.min(zs)

def draw_slice(
    ax, df,
    is_slice =True, scolor=[[1,0,0]], salpha=[0.1],
    is_border=True, bcolor=[[1,0,0]], balpha=[0.1],
    is_origin=True, ocolor=[[1,0,0]], oalpha=[0.1],
    is_arrow=True,
):
    df = df.copy()
    df = df.reset_index(drop=True)

    D = len(df)
    if len(scolor)==1: scolor = scolor*D
    if len(salpha)==1: salpha = salpha*D
    if len(bcolor)==1: bcolor = bcolor*D
    if len(balpha)==1: balpha = balpha*D
    if len(ocolor)==1: ocolor = bcolor*D
    if len(oalpha)==1: oalpha = balpha*D


    #for i,d in df.iterrows():
    for i in range(D):
        d = df.iloc[i]
        W, H = d.W, d.H
        o0, o1, o2, o3, o4, o5 = d.ImageOrientationPatient
        ox = np.array([o0, o1, o2])
        oy = np.array([o3, o4, o5])
        sx, sy, sz = d.ImagePositionPatient
        s = np.array([sx, sy, sz])
        delx, dely = d.PixelSpacing

        p0 = s
        p1 = s + W * delx * ox
        p2 = s + H * dely * oy
        p3 = s + H * dely * oy + W * delx * ox

        grid = np.stack([p0, p1, p2, p3]).reshape(2, 2, 3)
        gx = grid[:, :, 0]
        gy = grid[:, :, 1]
        gz = grid[:, :, 2]

        #outline
        if is_slice:
            ax.plot_surface(gx, gy, gz, color=scolor[i], alpha=salpha[i])

        if is_border:
            line = np.stack([p0, p1, p3, p2] )
            ax.plot(line[:,0], line[:,1], zs=line[:,2], color=ocolor[i], alpha=oalpha[i])

        if is_origin:
            ax.scatter([sx], [sy], [sz],   color=ocolor[i], alpha=oalpha[i])

    #check ordering of slice
    if is_arrow :
        sx0, sy0, sz0 = df.iloc[0].ImagePositionPatient
        sx1, sy1, sz1 = df.iloc[-1].ImagePositionPatient
        arrow_prop_dict = dict(mutation_scale=20, arrowstyle='-|>', color='k', shrinkA=0, shrinkB=0)
        a = Arrow3D([sx0, sx1], [sy0, sy1], [sz0, sz1], **arrow_prop_dict)
        ax.add_artist(a)


plot = False        


from tqdm import tqdm
dfs = []
error_ids = []
all_dfs = []
for study_id_n, study_id in enumerate(tqdm(id_df.study_id.unique())):
    for axial_t2_id in id_df[(id_df.study_id == study_id) & (id_df.series_description=='Axial T2')].series_id.values:
        try:

            sagittal_t2_id = id_df[(id_df.study_id == study_id) & (id_df.series_description=='Sagittal T2/STIR')].iloc[0].series_id
            data = read_study(study_id, axial_t2_id=axial_t2_id, sagittal_t2_id=sagittal_t2_id)


            #--- step.1 : detect 2d point in sagittal_t2
            sagittal_t2 = data.sagittal_t2.volume
            sagittal_t2_df = data.sagittal_t2.df
            axial_t2_df = data.axial_t2.df
            all_dfs.append(axial_t2_df)

            D,H,W = sagittal_t2.shape
            image = resize_volume(sagittal_t2,cfg.point_net.image_size)

            sagittal_t2_z = D//2
            image = image[sagittal_t2_z] #we use only center image #todo: better selection

            batch = dotdict(
                sagittal=torch.from_numpy(image).unsqueeze(0).unsqueeze(0).byte()
            )
            with torch.cuda.amp.autocast(enabled=True):
                with torch.no_grad():
                    output = point_net(batch)

            probability = output['probability'][0].float().data.cpu().numpy()
            sagittal_t2_point = probability_to_point(probability) #5 level 2d points, todo: check invalid output point

#             #for debug and development
#             point_hat, z_hat = sagittal_t2_point_hat = get_true_sagittal_t2_point(study_id, sagittal_t2_df)
#             point_hat = point_hat*[[cfg.point_net.image_size/W, cfg.point_net.image_size/H]]


            #--- step.2 : perdict slice level of axial_t2
            world_point = world_point = view_to_world(sagittal_t2_point, sagittal_t2_z, sagittal_t2_df, cfg.point_net.image_size)
            assigned_level, closest_z, closest_fdis = axial_t2_level = point_to_level(world_point, axial_t2_df)
            
            axial_t2_df['level'] = assigned_level
#             break
            exist_closest_fdis = closest_fdis[np.unique([c for c in assigned_level if c != 0])-1]
            exist_closest_z = closest_z[np.unique([c for c in assigned_level if c != 0])-1]
            ns = axial_t2_df.iloc[exist_closest_z].instance_number    
            axial_t2_df.loc[~axial_t2_df.instance_number.isin(ns), 'closest'] = 0
            axial_t2_df.loc[axial_t2_df.instance_number.isin(ns), 'closest'] = 1
            axial_t2_df['closest'] = axial_t2_df['closest'].astype(int)
            for level in range(1, 6):
                axial_t2_df.loc[axial_t2_df.level == level, 'dis'] = closest_fdis[level-1]            
                
            for n, dis in zip(ns, exist_closest_fdis):
                axial_t2_df.loc[axial_t2_df.instance_number == n, 'dis'] = dis
                
            assert len(assigned_level)==len(axial_t2_df)
            dfs.append(axial_t2_df)        



            if plot:
#             if exist_closest_fdis.max() > 5:                
                print(axial_t2_id, exist_closest_fdis.max(), assigned_level)
                ###################################################################
                #visualisation

                # https://matplotlib.org/stable/gallery/mplot3d/mixed_subplots.html
                fig = plt.figure(figsize=(23, 6))
                ax1 = fig.add_subplot(1, 1, 1)
                ax2 = fig.add_subplot(1, 2, 2, projection='3d')

                # detection result
                p = probability_to_rgb(probability)
                m = cv2.cvtColor(image, cv2.COLOR_GRAY2BGR)
                m = 255 - (255 - m * 0.8) * (1 - p / 255)

                ax1.imshow(m / 255)
                ax1.set_title(f'sagittal keypoint detection (unet)\n series_id: {axial_t2_id}')


                # draw  assigned_level
                level_ncolor = np.array(level_color) / 255
                coloring = level_ncolor[assigned_level].tolist()
                draw_slice(
                    ax2, axial_t2_df,
                    is_slice=True,   scolor=coloring, salpha=[0.1],
                    is_border=True,  bcolor=coloring, balpha=[0.2],
                    is_origin=False, ocolor=[[0, 0, 0]], oalpha=[0.0],
                    is_arrow=True
                )

            #     draw world_point
                ax2.scatter(world_point[:, 0], world_point[:, 1], world_point[:, 2], alpha=1, color='black')


                ### draw closest slice
                coloring = level_ncolor[1:].tolist()
                draw_slice(
                    ax2, axial_t2_df.iloc[closest_z],
                    is_slice=True, scolor=coloring, salpha=[0.1],
                    is_border=True, bcolor=coloring, balpha=[1],
                    is_origin=False, ocolor=[[1, 0, 0]], oalpha=[0],
                    is_arrow=False
                )

                ax2.set_aspect('equal')
                ax2.set_title(f'axial slice assignment\n series_id:{sagittal_t2_id}')
                ax2.set_xlabel('x')
                ax2.set_ylabel('y')
                ax2.set_zlabel('z')
                ax2.view_init(elev=0, azim=-10, roll=0)
                plt.tight_layout(pad=2)
                plt.show()
    
        except:
            error_ids.append(axial_t2_id)
axial_closest_df = pd.concat(dfs)
axial_direction = pd.concat(all_dfs)
axial_direction.to_csv('axial_direction.csv', index=False)
axial_closest_df.to_csv('axial_closest_df.csv', index=False)

ModuleNotFoundError: No module named '_dir_setting_'

# 2, setup

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
from glob import glob
import albumentations as A
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import copy
from scipy.special import softmax
def sigmoid(x):
    return 1/(1 + np.exp(-x))
from sklearn.metrics import roc_auc_score, confusion_matrix, mean_squared_error, average_precision_score, recall_score
import warnings
warnings.simplefilter('ignore')

import os
import cv2
import gdcm
import pydicom
import zipfile
import seaborn as sns
import matplotlib.pyplot as plt
import dicomsdl
from pdb import set_trace as st
from tqdm import tqdm
from joblib import Parallel, delayed
from pydicom.pixel_data_handlers.util import apply_voi_lut
import torch

In [ ]:
def standardize_pixel_array(dcm: pydicom.dataset.FileDataset) -> np.ndarray:
    """
    Source : https://www.kaggle.com/competitions/rsna-2023-abdominal-trauma-detection/discussion/427217
    """
    # Correct DICOM pixel_array if PixelRepresentation == 1.
    pixel_array = dcm.pixel_array
    if dcm.PixelRepresentation == 1:
        bit_shift = dcm.BitsAllocated - dcm.BitsStored
        dtype = pixel_array.dtype 
        new_array = (pixel_array << bit_shift).astype(dtype) >>  bit_shift
        pixel_array = pydicom.pixel_data_handlers.util.apply_modality_lut(new_array, dcm)
    return pixel_array

def get_center_x_path(study_id, series_id):
    path_x_map = {}
    for dcm_path in sorted(glob(f"{dicom_dir}/{study_id}/{series_id}/*.dcm")):
        filename = dcm_path.split('/')[-1].replace('.dcm', '')
    #         if int(instance_number) % 2 == 1:
    #             continue
        dicom = dicomsdl.open(dcm_path)
        pos_x = dicom['ImagePositionPatient'][0]
        path_x_map[pos_x] = dcm_path
        xs = []
        for i, k in enumerate(sorted(path_x_map.keys())):
            xs.append(k)
    return path_x_map[xs[len(xs)//2-1]], path_x_map[xs[len(xs)//2]], path_x_map[xs[len(xs)//2+1]]

def read_sagittal_x_center_dicom(args, verbose=False):
    study_id, series_id = args
    ch_imgs = []
    dcm_path_3 = get_center_x_path(study_id, series_id)
    for dcm_path in dcm_path_3:
        dicom = dicomsdl.open(dcm_path)
        img = dicom.pixelData(storedvalue = True)

        if dicom['PixelRepresentation'] == 1:
            bit_shift = dicom['BitsAllocated'] - dicom['BitsStored']
            dtype = img.dtype
            img = (img << bit_shift).astype(dtype) >>  bit_shift
        img = img.astype(np.float32)

        intercept = dicom['RescaleIntercept']
        slope = dicom['RescaleSlope']
        if (slope is not None) & (intercept is not None):
            img = img * slope + intercept

        img = (img - img.min()) / (img.max() - img.min() + 1e-6)

        if dicom.PhotometricInterpretation == "MONOCHROME1":
            img = 1 - img
        img = (img*255.0).astype('uint8')

        ch_imgs.append(img)
    if verbose:
        plt.imshow(ch_imgs[1], 'gray')
        plt.show()
    img = np.array(ch_imgs).transpose((1,2,0))
    cv2.imwrite(f'/kaggle/temp/sagittal_all_images/{study_id}___{series_id}.png', img)
    
    
    
def select_path_list(lst, N, offset=0, skip=1):
    if not lst:
        return [''] * (2 * N + 1)
    
    center = (len(lst) // 2) + offset
    result = [''] * (2 * N + 1)
    
    if 0 <= center < len(lst):
        result[N] = lst[center]
    
    for i in range(1, N + 1):
        left_index = center - i*skip
        right_index = center + i*skip
        
        if 0 <= left_index < len(lst):
            result[N - i] = lst[left_index]
        else:
            result[N - i] = lst[0]
        if 0 <= right_index < len(lst):
            result[N + i] = lst[right_index]
        else:
            result[N + i] = lst[len(lst)-1]
    
    return result

    
def read_sagittal_dicom(args, verbose=False):
    study_id, series_id = args
    imgs = {}
    origin_paths = []
    zs = []
    xyzs = []
    paths = []
    for dcm_path in glob(f"{dicom_dir}/{study_id}/{series_id}/*.dcm"):
        filename = dcm_path.split('/')[-1].replace('.dcm', '')
        dicom = dicomsdl.open(dcm_path)
        img = dicom.pixelData(storedvalue = True)

        if dicom['PixelRepresentation'] == 1:
            bit_shift = dicom['BitsAllocated'] - dicom['BitsStored']
            dtype = img.dtype
            img = (img << bit_shift).astype(dtype) >>  bit_shift
        img = img.astype(np.float32)

        intercept = dicom['RescaleIntercept']
        slope = dicom['RescaleSlope']
        if (slope is not None) & (intercept is not None):
            img = img * slope + intercept

        img = (img - img.min()) / (img.max() - img.min() + 1e-6)

        if dicom.PhotometricInterpretation == "MONOCHROME1":
            img = 1 - img
        img = (img*255.0).astype('uint8')
        save_path = f'/kaggle/temp/sagittal_all_images/{study_id}___{series_id}___{filename}.png'
        paths.append(save_path)

        imgs[save_path] = img
        origin_paths.append(dcm_path)
        xyzs.append(dicom['ImagePositionPatient'])

    df = pd.DataFrame({
        'path': paths,
        'origin_path': origin_paths,
    })
    df[['x_pos', 'y_pos', 'z_pos']] = np.array(xyzs)
    df['study_id'] = study_id
    df['series_id'] = series_id
    df['instance_number'] = df['path'].apply(lambda x: int(x.split('___')[-1].replace('.png', '')))
    df = df.sort_values(['x_pos', 'instance_number'])
    df = df.drop_duplicates('x_pos')        
    path_list = df.path.values
    for path_n, path in enumerate(df.path):
        if path_n == 0:
            prev_path = df.path.values[0]
        else:
            prev_path = df.path.values[path_n-1]
        prev_im = imgs[prev_path]            
        
        im = imgs[path]
        if path_n == len(df)-1:
            next_path = df.path.values[-1]
        else:
            next_path = df.path.values[path_n+1]
        next_im = imgs[next_path]
  
        if not (prev_im.shape == im.shape == next_im.shape):
            s = prev_im.shape
            im = cv2.resize(im, s)
            next_im = cv2.resize(next_im, s)

        image = np.array([prev_im, im, next_im]).transpose((1,2,0))
        cv2.imwrite(path, image)
    
    if verbose:
        rows = 7
        for n, k in enumerate(sorted(imgs.keys())):
            if n % rows == 0:
                fig = plt.figure(figsize=(20, 20))
            fig.add_subplot(1, rows, n%rows+1)

            im = imgs[k]
            plt.imshow(im, 'gray')

            if n % rows == rows-1:
                plt.show()        
        
    return df

def read_axial_dicom(args, verbose=False):
    study_id, series_id = args
    imgs = {}
    origin_paths = []
    zs = []
    xyzs = []
    paths = []
    series_axial_direction = axial_direction[axial_direction.series_id==series_id]
    for dcm_path in glob(f"{dicom_dir}/{study_id}/{series_id}/*.dcm"):
        filename = dcm_path.split('/')[-1].replace('.dcm', '')
#         if int(instance_number) % 2 == 1:
#             continue
        dicom = dicomsdl.open(dcm_path)
        img = dicom.pixelData(storedvalue = True)

        if dicom['PixelRepresentation'] == 1:
            bit_shift = dicom['BitsAllocated'] - dicom['BitsStored']
            dtype = img.dtype
            img = (img << bit_shift).astype(dtype) >>  bit_shift
        img = img.astype(np.float32)

        intercept = dicom['RescaleIntercept']
        slope = dicom['RescaleSlope']
        if (slope is not None) & (intercept is not None):
            img = img * slope + intercept

        pos_z = dicom['ImagePositionPatient'][-1]
        
        img = (img - img.min()) / (img.max() - img.min() + 1e-6)

        if dicom.PhotometricInterpretation == "MONOCHROME1":
            img = 1 - img
        img = (img*255.0).astype('uint8')
        save_path = f'/kaggle/temp/axial_all_images/{study_id}___{series_id}___{filename}.png'
        paths.append(save_path)

        imgs[save_path] = img
        origin_paths.append(dcm_path)
        xyzs.append(dicom['ImagePositionPatient'])

    df = pd.DataFrame({
        'path': paths,
        'origin_path': origin_paths,
    })
    df[['x_pos', 'y_pos', 'z_pos']] = np.array(xyzs)
    df['study_id'] = study_id
    df['series_id'] = series_id
    df['instance_number'] = df['path'].apply(lambda x: int(x.split('___')[-1].replace('.png', '')))

    l = len(df)
    if len(df.merge(series_axial_direction[['instance_number', 'z']], on='instance_number')) == l:
        df = df.merge(series_axial_direction[['instance_number', 'z']], on='instance_number')
        df = df.sort_values(['z', 'instance_number'])
    else:
        df = df.sort_values(['z_pos', 'instance_number'])
    path_list = df.path.values

    for path_n, path in enumerate(df.path):
        im = imgs[path]
        if path_n == 0:
            prev_path = df.path.values[0]
        else:
            prev_path = df.path.values[path_n-1]
        prev_im = imgs[prev_path]
        
        if path_n == len(df)-1:
            next_path = df.path.values[-1]
        else:
            next_path = df.path.values[path_n+1]
        next_im = imgs[next_path]
  
        if not (prev_im.shape == im.shape == next_im.shape):

            s = prev_im.shape
            im = cv2.resize(im, s)
            next_im = cv2.resize(next_im, s)

        image = np.array([prev_im, im, next_im]).transpose((1,2,0))
        cv2.imwrite(path, image)

    if series_id not in axial_closest_df.series_id.unique():
        for path_n, path in enumerate(df.path):
            im = imgs[path]
            if path_n <= 1:
                prev_path = df.path.values[0]
            else:
                prev_path = df.path.values[path_n-2]
            prev_im = imgs[prev_path]
            
            if path_n >= len(df)-2:
                next_path = df.path.values[-1]
            else:
                next_path = df.path.values[path_n+2]
            next_im = imgs[next_path]
      
            if not (prev_im.shape == im.shape == next_im.shape):

                s = prev_im.shape
                im = cv2.resize(im, s)
                next_im = cv2.resize(next_im, s)

            image = np.array([prev_im, im, next_im]).transpose((1,2,0))
            cv2.imwrite(path.replace('.png', '_stride2.png'), image)

        for path_n, path in enumerate(df.path):
            im = imgs[path]
            if path_n <= 2:
                prev_path = df.path.values[0]
            else:
                prev_path = df.path.values[path_n-3]
            prev_im = imgs[prev_path]
            
            if path_n >= len(df)-3:
                next_path = df.path.values[-1]
            else:
                next_path = df.path.values[path_n+3]
            next_im = imgs[next_path]
      
            if not (prev_im.shape == im.shape == next_im.shape):

                s = prev_im.shape
                im = cv2.resize(im, s)
                next_im = cv2.resize(next_im, s)

            image = np.array([prev_im, im, next_im]).transpose((1,2,0))
            cv2.imwrite(path.replace('.png', '_stride3.png'), image)
        
    if verbose:
        rows = 7
        for n, k in enumerate(sorted(imgs.keys())):
            if n % rows == 0:
                fig = plt.figure(figsize=(20, 20))
            fig.add_subplot(1, rows, n%rows+1)

            im = imgs[k]
            plt.imshow(im, 'gray')

            if n % rows == rows-1:
                plt.show()        
        
    return df


In [6]:
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

use_local_box_when_debug = False

if debug:
    df = pd.read_csv('/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/train_series_descriptions.csv')
    fold_df = pd.read_csv('/kaggle/input/rsna2024-csvs/train_v3.csv')
    df = df[df.study_id.isin(fold_df[fold_df.fold==debug_fold].study_id)]
    if short_debug:
        df = df[df.study_id.isin(use_ids)]
#         df = df[df.series_id==3636216534]

else:
    df = pd.read_csv('/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/test_series_descriptions.csv')
dicom_dir = f'/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/{train_test}_images'


NameError: name 'torch' is not defined

# dicom to png

In [5]:

image_save_dir = '/kaggle/temp/axial_all_images'
os.makedirs(image_save_dir, exist_ok=True)

axial_df = df[df.series_description == 'Axial T2']
args = axial_df.drop_duplicates(['study_id', 'series_id'])[['study_id', 'series_id']].values


from multiprocessing import Pool

p = Pool(processes=4)
results = []
with tqdm(total=len(args)) as pbar:
    for res in p.imap(read_axial_dicom, args):
        results.append(res)
        pbar.update(1)
p.close()

axial_df = pd.concat(results)
axial_df.to_csv('axial_df.csv', index=False)

image_save_dir = '/kaggle/temp/sagittal_all_images'
os.makedirs(image_save_dir, exist_ok=True)

sagittal_df = df[df.series_description != 'Axial T2']
args = sagittal_df.drop_duplicates(['study_id', 'series_id'])[['study_id', 'series_id']].values

from multiprocessing import Pool

p = Pool(processes=4)
results = []
with tqdm(total=len(args)) as pbar:
    for res in p.imap(read_sagittal_dicom, args):
        results.append(res)
        pbar.update(1)
p.close()

sagittal_df = pd.concat(results)
sagittal_df.to_csv('sagittal_df.csv', index=False)

sagittal_df = pd.read_csv('sagittal_df.csv')
series_description_df = pd.read_csv(f'/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/{train_test}_series_descriptions.csv')
sagittal_df = sagittal_df.merge(series_description_df, on=['study_id', 'series_id'])

for study_id_n, (study_id, idf) in enumerate(tqdm(sagittal_df.groupby('study_id'))):
    t1 = idf[idf.series_description=='Sagittal T1'].sort_values('instance_number')
    if len(t1)!=0:
        t1 = t1[t1.series_id==t1.series_id.values[0]]
    t2 = idf[idf.series_description=='Sagittal T2/STIR'].sort_values('instance_number')
    if len(t2)!=0:
        t2 = t2[t2.series_id==t2.series_id.values[0]]
    if len(t1)==0:
        m = t2.instance_number.max()
        mi = t2.instance_number.min()
    elif (len(t2)==0):
        m = t1.instance_number.max()
        mi = t1.instance_number.min()
    else:
        m = max([t1.instance_number.max(), t2.instance_number.max()])
        mi = min([t1.instance_number.min(), t2.instance_number.min()])
    for n in range(mi, m+1):
        n1 = t1[t1.instance_number == n]
        n2 = t2[t2.instance_number == n]
        if len(n1) != 0:
            # print(n1.path.values[0])
            im1 = cv2.imread(n1.path.values[0])[:,:,1]
        else:
            im1 = None
        if len(n2) != 0:
            # print(n2.path.values[0])
            im2 = cv2.imread(n2.path.values[0])[:,:,1]
        else:
            im2 = None
        if im1 is None:
            # raise
            im1 = np.zeros(im2.shape)
        if im2 is None:
            # raise
            im2 = np.zeros(im1.shape)
        if im1.shape!=im2.shape:
            im1 = cv2.resize(im1, (im2.shape[1], im2.shape[0]))
        im = np.array([im1, im2, im1]).transpose((1,2,0))

        cv2.imwrite(f'/kaggle/temp/sagittal_all_images/{study_id}___{n}.png', im)


NameError: name 'df' is not defined

In [8]:
axial_closest_df = axial_closest_df[axial_closest_df.closest==1]
axial_closest_df['pred_level'] = axial_closest_df.level.values
axial_closest_df['path'] = '/kaggle/temp/axial_all_images/' + axial_closest_df.study_id.astype(str) + '___' + axial_closest_df.series_id.astype(str) + '___' + axial_closest_df.instance_number.astype(str) + '.png'
axial_closest_df['level_pred'] = 1
axial_closest_df[['study_id', 'series_id', 'path', 'level_pred', 'pred_level', 'instance_number', 'z']]
axial_all_level_df = axial_closest_df[['study_id', 'series_id', 'path', 'level_pred', 'pred_level', 'instance_number', 'z', 'dis']]
axial_all_level_df.to_csv('axial_all_level_df.csv', index=False)

## stage 2:

In [9]:
import albumentations as A
from albumentations.pytorch import ToTensorV2
import timm

class RSNA2024DatasetV1(Dataset):
    def __init__(self, cfg):
        self.transforms = cfg.transforms
        self.cfg = cfg
        self.paths = cfg.df.path.values
        if 'x_min' in list(cfg.df):
            self.boxes = cfg.df[['x_min', 'y_min', 'x_max', 'y_max']].values
            
    def __len__(self):
        return len(self.paths)

    def __getitem__(self, idx):
        path = self.paths[idx]
        image = cv2.imread(path)[:,:,::-1]

        if hasattr(self.cfg, 'box_crop') and self.cfg.box_crop:
            box = self.boxes[idx]
            x_pad = (box[2] - box[0])//2 * self.cfg.box_crop_x_ratio
            y_pad = (box[3] - box[1])//2 * self.cfg.box_crop_y_ratio
            x_min = np.max([box[0]-x_pad, 0])
            y_min = np.max([box[1]-y_pad, 0])
            if hasattr(self.cfg, 'box_crop_y_upper_ratio'):
                y_upper_pad = (box[3] - box[1])//2 * self.cfg.box_crop_y_upper_ratio
                y_min = np.max([box[1]-y_upper_pad, 0])
            x_max = np.min([box[2]+x_pad, image.shape[1]])
            y_max = np.min([box[3]+y_pad, image.shape[0]])
            s = image.shape
            image = image[int(y_min):int(y_max), int(x_min):int(x_max), :]
            # print(s, image.shape)

        image = self.transforms(image=image)['image']

        return image

class base():
    def __init__(self):
        self.model_name = 'convnext_small.in12k_ft_in1k_384'
        self.image_size = 384
        self.batch_size = 16
        self.tta = False
        self.box_crop = None
        self.transforms = A.Compose([
            A.Resize(self.image_size, self.image_size),
            A.Normalize(),
            ToTensorV2(),
        ])
        
class rsna_sagittal_cl(base):
    def __init__(self):
        super().__init__()
        self.df = pd.read_csv('sagittal_df.csv')
        self.label_features = ['l1_spinal', 'l2_spinal', 'l3_spinal', 'l4_spinal', 'l5_spinal',
                               'l1_right_neural', 'l2_right_neural', 'l3_right_neural', 'l4_right_neural', 'l5_right_neural',
                               'l1_left_neural', 'l2_left_neural', 'l3_left_neural', 'l4_left_neural', 'l5_left_neural']
        self.image_size = 256
        
        d = '/kaggle/input/rsna-2024-sagittal-models-v1/rsna-sagittal-level-cl-spinal-nfn-v2'
        self.model = timm.create_model(self.model_name, pretrained=False, num_classes=len(self.label_features))
        if debug:
            self.model_paths = [f'{d}/last_fold{debug_fold}.ckpt' for fold in range(5)]
        else:
            self.model_paths = [f'{d}/last_fold{fold}.ckpt' for fold in range(5)]
        self.transforms = A.Compose([
            A.Compose([
                A.Resize(self.image_size, self.image_size),
                A.Normalize(),
                ToTensorV2(),
            ])
        ])              



In [10]:
all_preds = []
for cfg in [
    rsna_sagittal_cl(),
]:
    models = []
    for model_path in cfg.model_paths:
        # print(model_path)
        state_dict = torch.load(model_path, map_location=torch.device('cpu'))
        model = copy.deepcopy(cfg.model)
        model.load_state_dict(state_dict)
        model.to(DEVICE)
        model.eval()
        models.append(model)

    ds = RSNA2024DatasetV1(cfg)
    loader = DataLoader(ds, batch_size=cfg.batch_size, shuffle=False, drop_last=False, num_workers=4)
    preds = []
    for images in tqdm(loader, smoothing=0):
        images = images.to(DEVICE)
        batch_preds = []
        for model in models:
            batch_preds.append(model(images).detach().cpu().numpy())

        preds += np.mean(batch_preds, axis=0).tolist()
    all_preds.append(preds)
fs = [f'pred_{col}' for col in cfg.label_features]
cfg.df[fs] = sigmoid(np.mean(all_preds, 0))
cfg.df.to_csv(f'sagittal_with_position_preds.csv', index=False)


sagittal = cfg.df.copy()
sagittal['pred_spinal'] = sagittal[['pred_l1_spinal', 'pred_l2_spinal', 'pred_l3_spinal', 'pred_l4_spinal', 'pred_l5_spinal']].mean(1)
sagittal['pred_right_neural'] = sagittal[['pred_l1_right_neural', 'pred_l2_right_neural', 'pred_l3_right_neural', 'pred_l4_right_neural', 'pred_l5_right_neural']].mean(1)
sagittal['pred_left_neural'] = sagittal[['pred_l1_left_neural', 'pred_l2_left_neural', 'pred_l3_left_neural', 'pred_l4_left_neural', 'pred_l5_left_neural']].mean(1)


series_description_df = pd.read_csv(f'/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/{train_test}_series_descriptions.csv')
t2_ids = series_description_df[series_description_df.series_description=='Sagittal T2/STIR'].series_id

dfs = []
for id, idf in sagittal[sagittal.series_id.isin(t2_ids)].groupby('series_id'):
    idf = idf.sort_values(['x_pos', 'instance_number'])
    idf = idf.drop_duplicates('x_pos')
    ldf = idf[idf['pred_spinal']==idf['pred_spinal'].max()].iloc[:1]
    dfs.append(ldf)

spinal_sagittal = pd.concat(dfs)
sagittal.to_csv('sagittal_df.csv', index=False)
spinal_sagittal.to_csv('spinal_sagittal_df.csv', index=False)
spinal_sagittal.study_id.nunique(), spinal_sagittal.series_id.nunique()

100%|██████████| 4/4 [00:03<00:00,  1.26it/s]


(1, 1)

### yolox

In [11]:
# find left
box_cols = ['x_min', 'y_min', 'x_max','y_max']
yolo_for_config_dir = '/kaggle/input/rsna-2024-axial-models-v1/rsna-axial-all-images-left-yolox-x'

yolo_for_config = 'rsna_axial_all_images_left_yolox_x'

import sys
sys.path.append('/kaggle/input/yolox20230421/YOLOX')

from yolox.utils import postprocess
from yolox.data.data_augment import ValTransform


class MyDataset(Dataset):
    def __init__(self, df):
        self.paths = df.path.values
        self.preproc = ValTransform(legacy = False)
        self.size = 512

    def __len__(self):
        return len(self.paths)

    def __getitem__(self, idx):
        path = self.paths[idx]
        img = cv2.imread(path)
        ratio = min(self.size / img.shape[0], self.size / img.shape[1])

        img, _ = self.preproc(img, None, (self.size, self.size))
        img = torch.from_numpy(img).float()
        img = img.float()

        return img, path, ratio

from yolox.exp import Exp as MyExp

class ExpX(MyExp):
    def __init__(self):
        super(ExpX, self).__init__()
        self.depth = 1.33
        self.width = 1.25
        self.exp_name = ''
        self.data_dir = ""

        ### need change ###
        self.max_epoch = 10
        self.output_dir = "."
        self.input_size = (512, 512)
        self.test_size = (512, 512)
        self.no_aug_epochs = 10 # 15
        self.warmup_epochs = 5 # 5
        self.num_classes = 1
        ### need change ###

        ### fyi ###
        self.data_num_workers = 16
        self.eval_interval = 1
        self.seed = 42
        self.print_interval = 100000
        self.eval_interval = 1
        self.save_history_ckpt = False
        self.mosaic_prob = 1.0
        self.mixup_prob = 1.0
        self.hsv_prob = 1.0
        self.flip_prob = 0.5
        self.degrees = 10.0
        self.translate = 0.1
        self.mosaic_scale = (0.1, 2)
        self.enable_mixup = True
        self.mixup_scale = (0.5, 1.5)
        self.shear = 2.0
        self.min_lr_ratio = 0.05
        self.basic_lr_per_img = 0.00015625
        self.scheduler = 'yoloxwarmcos'
        self.ema = True
        self.weight_decay = 0.0005
        self.momentum = 0.9
        self.test_conf = 0.01
        self.nmsthre = 0.65
        self.class_id_name_map = {0: 'left'}

class ExpL(MyExp):
    def __init__(self):
        super(ExpL, self).__init__()
        self.depth = 1.0
        self.width = 1.0
        self.exp_name = ''
        self.data_dir = ""

        ### need change ###
        self.max_epoch = 10
        self.output_dir = "."
        self.input_size = (512, 512)
        self.test_size = (512, 512)
        self.no_aug_epochs = 10 # 15
        self.warmup_epochs = 5 # 5
        self.num_classes = 1
        ### need change ###

        ### fyi ###
        self.data_num_workers = 16
        self.eval_interval = 1
        self.seed = 42
        self.print_interval = 100000
        self.eval_interval = 1
        self.save_history_ckpt = False
        self.mosaic_prob = 1.0
        self.mixup_prob = 1.0
        self.hsv_prob = 1.0
        self.flip_prob = 0.5
        self.degrees = 10.0
        self.translate = 0.1
        self.mosaic_scale = (0.1, 2)
        self.enable_mixup = True
        self.mixup_scale = (0.5, 1.5)
        self.shear = 2.0
        self.min_lr_ratio = 0.05
        self.basic_lr_per_img = 0.00015625
        self.scheduler = 'yoloxwarmcos'
        self.ema = True
        self.weight_decay = 0.0005
        self.momentum = 0.9
        self.test_conf = 0.01
        self.nmsthre = 0.65
        self.class_id_name_map = {0: 'left'}

exp_x = ExpX()

# set inference parameters
confthre = 0.03
nmsthre = 0.45

# get YOLOX model
models = []
for fold in range(1):
    model = exp_x.get_model()
    model.to(DEVICE)
    model.eval()
    model.head.training=False
    model.training=False
    ckpt_file = "/kaggle/input/notebookfa5fb155d5/rsna-axial-all-images-left-yolox-x/0_best_ckpt.ckpt"
    # print(ckpt_file)
    ckpt = torch.load(ckpt_file, map_location="cpu")
    model.load_state_dict(ckpt["model"])
    models.append(model)

ds = MyDataset(axial_all_level_df)
loader = DataLoader(ds, batch_size=16, shuffle=False, drop_last=False, num_workers=4)

all_preds_list = [[]]
all_ratios_list = [[]]
all_paths_list = [[]]

# print('inf start...')
with torch.no_grad():
    for loader_n, input in tqdm(enumerate(loader)):
        # if loader_n % 100 == 0:
        #     print(loader_n)
        images, paths, ratios = input
        images = images.to(DEVICE)
        for model_n, model in enumerate(models):
            outputs = model(images)
#             outputs2 = postprocess(
#                         outputs, exp_x.num_classes, confthre,
#                         nmsthre, class_agnostic=True
#                     )
            
#             st()
            
            outputs = postprocess(
                        outputs, exp_x.num_classes, confthre,
                        nmsthre, class_agnostic=True
                    )
            all_preds_list[model_n] += outputs
            all_ratios_list[model_n] += list(ratios)
            all_paths_list[model_n] += list(paths)

df_5models = []
for model_n, (all_preds, all_paths, all_ratios) in enumerate(zip(all_preds_list, all_paths_list, all_ratios_list)):
    dfs = []
    all_boxes = []
    all_class_ids = []
    all_scores = []
    for n, (predictions, path, ratio)  in enumerate(zip(all_preds, all_paths, all_ratios)):
        if predictions is None:
            continue
        predictions = predictions.cpu().numpy()

        bboxes = predictions[:, 0:4]

        bboxes /= ratio
        bboxes = bboxes.tolist()
        bbclasses = predictions[:, 6]
        scores = predictions[:, 4] * predictions[:, 5]
        path_df = axial_all_level_df[axial_all_level_df.path==path].iloc[:1]
        for box, score, class_id in zip(bboxes, scores, bbclasses):
            all_boxes.append(box)
            all_scores.append(score)
            all_class_ids.append(class_id)
            dfs.append(path_df)
    tmp = pd.concat(dfs)
    tmp['class_id'] = all_class_ids
    tmp['class_id'] = tmp['class_id'].astype(int)
    tmp['class_name'] = tmp['class_id'].map(exp_x.class_id_name_map)
    tmp['conf'] = all_scores
    tmp[box_cols] = all_boxes
    tmp[box_cols] = np.round(tmp[box_cols]).astype(int)
    tmp['model_n'] = model_n
    df_5models.append(tmp)

from ensemble_boxes import *
from multiprocessing import cpu_count
import copy

from multiprocessing import Pool

def exec(args):
    path, path_df = args
    boxes_list = []
    confs_list = []
    labels_list = []
    for _, model_df in path_df.groupby('model_n'):
        boxes_list.append(model_df[box_cols].values/max_value)
        confs_list.append(model_df['conf'].values.tolist())
        labels_list.append(model_df['class_id'].values.tolist())
    boxes, confs, labels = weighted_boxes_fusion(boxes_list, confs_list, labels_list, weights=[1]*len(boxes_list), iou_thr=iou_thr, skip_box_thr=skip_box_thr)
    boxes *= max_value
    results = []
    for idx, box in enumerate(boxes):
        results.append({
            "path": path,
            "class_id": int(labels[idx]),
            'conf':confs[idx],
            "x_min": box[0],
            "y_min": box[1],
            "x_max": box[2],
            "y_max": box[3],
        })
    return results

result_oof = copy.deepcopy(df_5models[0])
yolo_pred_df = pd.concat(df_5models)
weights = [1]* len(df_5models)
iou_thr = 0.4
min_conf = yolo_pred_df.conf.min()
skip_box_thr = 0.0001
results = []
max_value = 12800

class_id_name_map = {}
tmp = result_oof.drop_duplicates('class_name')
for class_id, class_name in zip(tmp.class_id.values, tmp.class_name.values):
    class_id_name_map[class_id] = class_name

wbf_result_maps_list = []
df_list = list(yolo_pred_df.groupby('path'))
p = Pool(processes=cpu_count())
with tqdm(total=len(df_list)) as pbar:
    for wbf_result_maps in p.imap(exec, df_list):
        wbf_result_maps_list += wbf_result_maps
        pbar.update(1)
p.close()

results = pd.DataFrame(wbf_result_maps_list)
for col in ['class_name', 'class_id', 'conf', 'x_min', 'y_min', 'x_max', 'y_max']:
    if col in list(result_oof):
        del result_oof[col]
results = results.merge(result_oof.drop_duplicates('path'), on='path')
results['class_name'] = results['class_id'].map(class_id_name_map)
del results['model_n']

dfs = []
for i, idf in results[['path', 'conf','class_id' ,'x_min','y_min','x_max','y_max']].groupby('path'):
    dfs.append(idf[idf.conf==idf.conf.max()].iloc[:1])
results = pd.concat(dfs)

results.to_csv(f'yolo_results_axial_left.csv', index=False)
results.head()

if plot:
    for path, pdf in results[results.path.isin(results.path.unique()[:5])].groupby('path'):
        pdf = pdf[pdf.conf==pdf.conf.max()].iloc[:1]
        im = cv2.imread(path)
        im = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
        for class_id, box in zip(pdf.class_id.values, pdf[box_cols].values):
            cv2.rectangle(
                im,
                pt1=(int(box[0]), int(box[1])),
                pt2=(int(box[2]), int(box[3])),
                color=(255,255,255),
                thickness=3
            )
        plt.imshow(im[:,:,0], 'gray')
        plt.show()

1it [00:01,  1.00s/it]
100%|██████████| 5/5 [00:00<00:00, 293.38it/s]


In [12]:
box_cols = ['x_min', 'y_min', 'x_max','y_max']
yolo_for_config_dir = '/kaggle/input/rsna-2024-axial-models-v1/rsna-axial-all-images-right-yolox-x'

yolo_for_config = 'rsna_axial_all_images_right_yolox_x'

import sys
sys.path.append('/kaggle/input/yolox20230421/YOLOX')

from yolox.utils import postprocess
from yolox.data.data_augment import ValTransform


class MyDataset(Dataset):
    def __init__(self, df):
        self.paths = df.path.values
        self.preproc = ValTransform(legacy = False)
        self.size = 512

    def __len__(self):
        return len(self.paths)

    def __getitem__(self, idx):
        path = self.paths[idx]
        img = cv2.imread(path)
        ratio = min(self.size / img.shape[0], self.size / img.shape[1])

        img, _ = self.preproc(img, None, (self.size, self.size))
        img = torch.from_numpy(img).float()
        img = img.float()

        return img, path, ratio

from yolox.exp import Exp as MyExp

class ExpX(MyExp):
    def __init__(self):
        super(ExpX, self).__init__()
        self.depth = 1.33
        self.width = 1.25
        self.exp_name = ''
        self.data_dir = ""

        ### need change ###
        self.max_epoch = 10
        self.output_dir = "."
        self.input_size = (512, 512)
        self.test_size = (512, 512)
        self.no_aug_epochs = 10 # 15
        self.warmup_epochs = 5 # 5
        self.num_classes = 1
        ### need change ###

        ### fyi ###
        self.data_num_workers = 16
        self.eval_interval = 1
        self.seed = 42
        self.print_interval = 100000
        self.eval_interval = 1
        self.save_history_ckpt = False
        self.mosaic_prob = 1.0
        self.mixup_prob = 1.0
        self.hsv_prob = 1.0
        self.flip_prob = 0.5
        self.degrees = 10.0
        self.translate = 0.1
        self.mosaic_scale = (0.1, 2)
        self.enable_mixup = True
        self.mixup_scale = (0.5, 1.5)
        self.shear = 2.0
        self.min_lr_ratio = 0.05
        self.basic_lr_per_img = 0.00015625
        self.scheduler = 'yoloxwarmcos'
        self.ema = True
        self.weight_decay = 0.0005
        self.momentum = 0.9
        self.test_conf = 0.01
        self.nmsthre = 0.65
        self.class_id_name_map = {0: 'right'}

class ExpL(MyExp):
    def __init__(self):
        super(ExpL, self).__init__()
        self.depth = 1.0
        self.width = 1.0
        self.exp_name = ''
        self.data_dir = ""

        ### need change ###
        self.max_epoch = 10
        self.output_dir = "."
        self.input_size = (512, 512)
        self.test_size = (512, 512)
        self.no_aug_epochs = 10 # 15
        self.warmup_epochs = 5 # 5
        self.num_classes = 1
        ### need change ###

        ### fyi ###
        self.data_num_workers = 16
        self.eval_interval = 1
        self.seed = 42
        self.print_interval = 100000
        self.eval_interval = 1
        self.save_history_ckpt = False
        self.mosaic_prob = 1.0
        self.mixup_prob = 1.0
        self.hsv_prob = 1.0
        self.flip_prob = 0.5
        self.degrees = 10.0
        self.translate = 0.1
        self.mosaic_scale = (0.1, 2)
        self.enable_mixup = True
        self.mixup_scale = (0.5, 1.5)
        self.shear = 2.0
        self.min_lr_ratio = 0.05
        self.basic_lr_per_img = 0.00015625
        self.scheduler = 'yoloxwarmcos'
        self.ema = True
        self.weight_decay = 0.0005
        self.momentum = 0.9
        self.test_conf = 0.01
        self.nmsthre = 0.65
        self.class_id_name_map = {0: 'right'}

exp_x = ExpX()

# set inference parameters
confthre = 0.03
nmsthre = 0.45

# get YOLOX model
models = []
for fold in range(1):
    model = exp_x.get_model()
    model.to(DEVICE)
    model.eval()
    model.head.training=False
    model.training=False
    ckpt_file = "/kaggle/input/notebookfa5fb155d5/rsna-axial-all-images-right-yolox-x/0_best_ckpt.ckpt"
    # print(ckpt_file)
    ckpt = torch.load(ckpt_file, map_location="cpu")
    model.load_state_dict(ckpt["model"])
    models.append(model)

ds = MyDataset(axial_all_level_df)
loader = DataLoader(ds, batch_size=16, shuffle=False, drop_last=False, num_workers=4)

all_preds_list = [[]]
all_ratios_list = [[]]
all_paths_list = [[]]

# print('inf start...')
with torch.no_grad():
    for loader_n, input in tqdm(enumerate(loader)):
        # if loader_n % 100 == 0:
        #     print(loader_n)
        images, paths, ratios = input
        images = images.to(DEVICE)
        for model_n, model in enumerate(models):
            outputs = model(images)
            outputs = postprocess(
                        outputs, exp_x.num_classes, confthre,
                        nmsthre, class_agnostic=True
                    )
            all_preds_list[model_n] += outputs
            all_ratios_list[model_n] += list(ratios)
            all_paths_list[model_n] += list(paths)

df_5models = []
for model_n, (all_preds, all_paths, all_ratios) in enumerate(zip(all_preds_list, all_paths_list, all_ratios_list)):
    dfs = []
    all_boxes = []
    all_class_ids = []
    all_scores = []
    for n, (predictions, path, ratio)  in enumerate(zip(all_preds, all_paths, all_ratios)):
        if predictions is None:
            continue
        predictions = predictions.cpu().numpy()

        bboxes = predictions[:, 0:4]

        bboxes /= ratio
        bboxes = bboxes.tolist()
        bbclasses = predictions[:, 6]
        scores = predictions[:, 4] * predictions[:, 5]
        path_df = axial_all_level_df[axial_all_level_df.path==path].iloc[:1]
        for box, score, class_id in zip(bboxes, scores, bbclasses):
            all_boxes.append(box)
            all_scores.append(score)
            all_class_ids.append(class_id)
            dfs.append(path_df)
    tmp = pd.concat(dfs)
    tmp['class_id'] = all_class_ids
    tmp['class_id'] = tmp['class_id'].astype(int)
    tmp['class_name'] = tmp['class_id'].map(exp_x.class_id_name_map)
    tmp['conf'] = all_scores
    tmp[box_cols] = all_boxes
    tmp[box_cols] = np.round(tmp[box_cols]).astype(int)
    tmp['model_n'] = model_n
    df_5models.append(tmp)




from ensemble_boxes import *
from multiprocessing import cpu_count
import copy

from multiprocessing import Pool

def exec(args):
    path, path_df = args
    boxes_list = []
    confs_list = []
    labels_list = []
    for _, model_df in path_df.groupby('model_n'):
        boxes_list.append(model_df[box_cols].values/max_value)
        confs_list.append(model_df['conf'].values.tolist())
        labels_list.append(model_df['class_id'].values.tolist())
    boxes, confs, labels = weighted_boxes_fusion(boxes_list, confs_list, labels_list, weights=[1]*len(boxes_list), iou_thr=iou_thr, skip_box_thr=skip_box_thr)
    boxes *= max_value
    results = []
    for idx, box in enumerate(boxes):
        results.append({
            "path": path,
            "class_id": int(labels[idx]),
            'conf':confs[idx],
            "x_min": box[0],
            "y_min": box[1],
            "x_max": box[2],
            "y_max": box[3],
        })
    return results

result_oof = copy.deepcopy(df_5models[0])
yolo_pred_df = pd.concat(df_5models)
weights = [1]* len(df_5models)
iou_thr = 0.4
min_conf = yolo_pred_df.conf.min()
skip_box_thr = 0.0001
results = []
max_value = 12800

class_id_name_map = {}
tmp = result_oof.drop_duplicates('class_name')
for class_id, class_name in zip(tmp.class_id.values, tmp.class_name.values):
    class_id_name_map[class_id] = class_name

wbf_result_maps_list = []
df_list = list(yolo_pred_df.groupby('path'))
p = Pool(processes=cpu_count())
with tqdm(total=len(df_list)) as pbar:
    for wbf_result_maps in p.imap(exec, df_list):
        wbf_result_maps_list += wbf_result_maps
        pbar.update(1)
p.close()

results = pd.DataFrame(wbf_result_maps_list)
for col in ['class_name', 'class_id', 'conf', 'x_min', 'y_min', 'x_max', 'y_max']:
    if col in list(result_oof):
        del result_oof[col]
results = results.merge(result_oof.drop_duplicates('path'), on='path')
results['class_name'] = results['class_id'].map(class_id_name_map)
del results['model_n']

dfs = []
for i, idf in results[['path', 'conf','class_id' ,'x_min','y_min','x_max','y_max']].groupby('path'):
    dfs.append(idf[idf.conf==idf.conf.max()].iloc[:1])
results = pd.concat(dfs)    

results.to_csv(f'yolo_results_axial_right.csv', index=False)
results.head()
if plot:

    for path, pdf in results[results.path.isin(results.path.unique()[:5])].groupby('path'):
        pdf = pdf[pdf.conf==pdf.conf.max()].iloc[:1]
        im = cv2.imread(path)
        im = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
        for class_id, box in zip(pdf.class_id.values, pdf[box_cols].values):
            cv2.rectangle(
                im,
                pt1=(int(box[0]), int(box[1])),
                pt2=(int(box[2]), int(box[3])),
                color=(255,255,255),
                thickness=3
            )
        plt.imshow(im[:,:,0], 'gray')
        plt.show()


left = pd.read_csv('yolo_results_axial_left.csv')
right = pd.read_csv('yolo_results_axial_right.csv')
for c in ['x_min', 'y_min', 'x_max', 'y_max']:
    right = right.rename(columns={c: 'right_'+c})
    left = left.rename(columns={c: 'left_'+c})
axial_box_df = right.merge(left[['path']+['left_x_min', 'left_y_min', 'left_x_max', 'left_y_max']], on='path')
axial_box_df['x_min'] = axial_box_df[['right_x_min', 'left_x_min']].min(1)
axial_box_df['y_min'] = axial_box_df[['right_y_min', 'left_y_min']].min(1)
axial_box_df['x_max'] = axial_box_df[['right_x_max', 'left_x_max']].max(1)
axial_box_df['y_max'] = axial_box_df[['right_y_max', 'left_y_max']].max(1)
axial_box_df.path.nunique(), len(axial_all_level_df)
if 'x_min' not in list(axial_all_level_df):
    axial_all_level_df = axial_all_level_df.merge(axial_box_df[['x_min', 'y_min', 'x_max', 'y_max', 'path']], on='path')


from multiprocessing import Pool

def exec(p):
    im=cv2.imread(p)
    return im.shape[:2]

p = Pool(processes=4)
args = axial_all_level_df.path.values
results = p.map(exec, args)    
image_size_df = pd.DataFrame(results)
image_size_df.columns = ['image_height', 'image_width']
image_size_df['path'] = args
if 'image_height' not in list(axial_all_level_df):
    axial_all_level_df = axial_all_level_df.merge(image_size_df, on='path')
axial_all_level_df = axial_all_level_df.sort_values(['series_id', 'pred_level'])    
axial_all_level_df.to_csv('axial_with_box_df.csv', index=False)    
axial_all_level_df.head(6)

1it [00:00,  2.07it/s]
100%|██████████| 5/5 [00:00<00:00, 205.60it/s]


,study_id,series_id,path,level_pred,pred_level,instance_number,z,dis,x_min,y_min,x_max,y_max,image_height,image_width
4,44036939,3481971518,/kaggle/temp/axial_all_images/44036939___34819...,1,1,17,30,0.818616,130.0,138.0,172.0,157.0,256,256
3,44036939,3481971518,/kaggle/temp/axial_all_images/44036939___34819...,1,2,23,24,0.454633,119.0,122.0,158.0,141.0,256,256
2,44036939,3481971518,/kaggle/temp/axial_all_images/44036939___34819...,1,3,29,18,0.247615,98.0,103.0,137.0,122.0,256,256
1,44036939,3481971518,/kaggle/temp/axial_all_images/44036939___34819...,1,4,34,13,0.592086,98.0,99.0,135.0,118.0,256,256
0,44036939,3481971518,/kaggle/temp/axial_all_images/44036939___34819...,1,5,39,8,0.332912,106.0,110.0,141.0,129.0,256,256


In [13]:
box_cols = ['x_min', 'y_min', 'x_max','y_max']
yolo_for_config_dir = '/kaggle/input/rsna-10classes-yolox-x'
yolo_for_config = 'rsna_10classes_yolox_x'

import sys
sys.path.append('/kaggle/input/yolox20230421/YOLOX')

from yolox.utils import postprocess
from yolox.data.data_augment import ValTransform

class MyDataset(Dataset):
    def __init__(self, df):
        self.paths = df.path.values
        self.preproc = ValTransform(legacy = False)
        self.size = 512

    def __len__(self):
        return len(self.paths)

    def __getitem__(self, idx):
        path = self.paths[idx]
        img = cv2.imread(path)
        ratio = min(self.size / img.shape[0], self.size / img.shape[1])

        img, _ = self.preproc(img, None, (self.size, self.size))
        img = torch.from_numpy(img).float()
        img = img.float()

        return img, path, ratio

from yolox.exp import Exp as MyExp

class Exp(MyExp):
    def __init__(self):
        super(Exp, self).__init__()
        self.depth = 1.33
        self.width = 1.25
        self.exp_name = ''
        self.data_dir = ""

        ### need change ###
        self.max_epoch = 10
        self.output_dir = "."
        self.input_size = (512, 512)
        self.test_size = (512, 512)
        self.no_aug_epochs = 10 # 15
        self.warmup_epochs = 5 # 5
        self.num_classes = 10
        ### need change ###

        ### fyi ###
        self.data_num_workers = 16
        self.eval_interval = 1
        self.seed = 42
        self.print_interval = 100
        self.eval_interval = 1
        self.save_history_ckpt = False
        self.mosaic_prob = 1.0
        self.mixup_prob = 1.0
        self.hsv_prob = 1.0
        self.flip_prob = 0.5
        self.degrees = 10.0
        self.translate = 0.1
        self.mosaic_scale = (0.1, 2)
        self.enable_mixup = True
        self.mixup_scale = (0.5, 1.5)
        self.shear = 2.0
        self.min_lr_ratio = 0.05
        self.basic_lr_per_img = 0.00015625
        self.scheduler = 'yoloxwarmcos'
        self.ema = True
        self.weight_decay = 0.0005
        self.momentum = 0.9
        self.test_conf = 0.01
        self.nmsthre = 0.65
        self.class_id_name_map = {
             0: 'L1/L2_L',
             1: 'L1/L2_R',
             2: 'L2/L3_L',
             3: 'L2/L3_R',
             4: 'L3/L4_L',
             5: 'L3/L4_R',
             6: 'L4/L5_L',
             7: 'L4/L5_R',
             8: 'L5/S1_L',
             9: 'L5/S1_R'
        }

exp = Exp()

# set inference parameters
confthre = 0.03
nmsthre = 0.45

# get YOLOX model
models = []
for fold in range(5):
    model = exp.get_model()
    model.cuda()
    model.eval()
    model.head.training=False
    model.training=False
    if debug:
        ckpt_file = f"/kaggle/input/notebookfa5fb155d5/rsna-10classes-yolox-x/{debug_fold}_best_ckpt.ckpt"
    else:
        ckpt_file = f"/kaggle/input/notebookfa5fb155d5/rsna-10classes-yolox-x/{fold}_best_ckpt.ckpt"
    # print(ckpt_file)
    ckpt = torch.load(ckpt_file, map_location="cpu")
    model.load_state_dict(ckpt["model"])
    models.append(model)
ds = MyDataset(spinal_sagittal)
loader = DataLoader(ds, batch_size=16, shuffle=False, drop_last=False, num_workers=4)

all_preds_list = [[], [], [], [], []]
all_ratios_list = [[], [], [], [], []]
all_paths_list = [[], [], [], [], []]

# print('inf start...')
with torch.no_grad():
    for loader_n, input in tqdm(enumerate(loader)):
        # if loader_n % 100 == 0:
        #     print(loader_n)
        images, paths, ratios = input
        images = images.cuda()
        for model_n, model in enumerate(models):
            outputs = model(images)
            outputs = postprocess(
                        outputs, exp.num_classes, confthre,
                        nmsthre, class_agnostic=True
                    )
            all_preds_list[model_n] += outputs
            all_ratios_list[model_n] += list(ratios)
            all_paths_list[model_n] += list(paths)

df_5models = []
for model_n, (all_preds, all_paths, all_ratios) in enumerate(zip(all_preds_list, all_paths_list, all_ratios_list)):
    dfs = []
    all_boxes = []
    all_class_ids = []
    all_scores = []
    for n, (predictions, path, ratio)  in enumerate(zip(all_preds, all_paths, all_ratios)):
        if predictions is None:
            continue
        predictions = predictions.cpu().numpy()

        bboxes = predictions[:, 0:4]

        bboxes /= ratio
        bboxes = bboxes.tolist()
        bbclasses = predictions[:, 6]
        scores = predictions[:, 4] * predictions[:, 5]
        path_df = sagittal_df[sagittal_df.path==path]
        for box, score, class_id in zip(bboxes, scores, bbclasses):
            all_boxes.append(box)
            all_scores.append(score)
            all_class_ids.append(class_id)
            dfs.append(path_df)
    tmp = pd.concat(dfs)
    tmp['class_id'] = all_class_ids
    tmp['class_id'] = tmp['class_id'].astype(int)
    tmp['class_name'] = tmp['class_id'].map(exp.class_id_name_map)
    tmp['conf'] = all_scores
    tmp[box_cols] = all_boxes
    tmp[box_cols] = np.round(tmp[box_cols]).astype(int)
    tmp['model_n'] = model_n
    df_5models.append(tmp)

from ensemble_boxes import *
from multiprocessing import cpu_count
import copy

from multiprocessing import Pool

def exec(args):
    path, path_df = args
    boxes_list = []
    confs_list = []
    labels_list = []
    for _, model_df in path_df.groupby('model_n'):
        boxes_list.append(model_df[box_cols].values/max_value)
        confs_list.append(model_df['conf'].values.tolist())
        labels_list.append(model_df['class_id'].values.tolist())
    boxes, confs, labels = weighted_boxes_fusion(boxes_list, confs_list, labels_list, weights=[1]*len(boxes_list), iou_thr=iou_thr, skip_box_thr=skip_box_thr)
    boxes *= max_value
    results = []
    for idx, box in enumerate(boxes):
        results.append({
            "path": path,
            "class_id": int(labels[idx]),
            'conf':confs[idx],
            "x_min": box[0],
            "y_min": box[1],
            "x_max": box[2],
            "y_max": box[3],
        })
    return results

result_oof = copy.deepcopy(df_5models[0])
yolo_pred_df = pd.concat(df_5models)
weights = [1]* len(df_5models)
iou_thr = 0.4
min_conf = yolo_pred_df.conf.min()
skip_box_thr = 0.0001
results = []
max_value = 12800

class_id_name_map = {}
tmp = result_oof.drop_duplicates('class_name')
for class_id, class_name in zip(tmp.class_id.values, tmp.class_name.values):
    class_id_name_map[class_id] = class_name

wbf_result_maps_list = []
df_list = list(yolo_pred_df.groupby('path'))
p = Pool(processes=cpu_count())
with tqdm(total=len(df_list)) as pbar:
    for wbf_result_maps in p.imap(exec, df_list):
        wbf_result_maps_list += wbf_result_maps
        pbar.update(1)
p.close()

results = pd.DataFrame(wbf_result_maps_list)
for col in ['class_name', 'class_id', 'conf', 'x_min', 'y_min', 'x_max', 'y_max']:
    if col in list(result_oof):
        del result_oof[col]
results = results.merge(result_oof.drop_duplicates('path'), on='path')
results['class_name'] = results['class_id'].map(class_id_name_map)
del results['model_n']

dfs = []
for i, idf in results.groupby(['path', 'class_id']):
    dfs.append(idf[idf.conf==idf.conf.max()].iloc[:1])
results = pd.concat(dfs)    


results.to_csv(f'yolo_results_sagittal.csv', index=False)
results.head()
if plot:

    for path, pdf in results[results.path.isin(results.path.unique()[:5])].groupby('path'):
        im = cv2.imread(path)
        im = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
        for class_id, box in zip(pdf.class_id.values, pdf[box_cols].values):
            cv2.rectangle(
                im,
                pt1=(int(box[0]), int(box[1])),
                pt2=(int(box[2]), int(box[3])),
                color=(255,255,255),
                thickness=3
            )
        plt.imshow(im[:,:,0], 'gray')
        plt.show()

del models
torch.cuda.empty_cache()


spinal_sagittal = pd.read_csv('spinal_sagittal_df.csv')
l = len(spinal_sagittal)
yolo_results_sagittal = pd.read_csv('yolo_results_sagittal.csv')[['path', 'class_id', 'class_name', 'conf','x_min','y_min','x_max','y_max']]
yolo_results_sagittal['study_id'] = yolo_results_sagittal.path.apply(lambda x: int(x.split('/')[-1].split('___')[0]))
del yolo_results_sagittal['path']

sagittal_box_df = spinal_sagittal.merge(yolo_results_sagittal, on='study_id')
sagittal_box_df['level'] = sagittal_box_df.class_name.apply(lambda x: x.split('_')[0])
sagittal_box_df['lr'] = sagittal_box_df.class_name.apply(lambda x: x.split('_')[1])
sagittal_box_df = sagittal_box_df[['study_id', 'path', 'level', 'lr']+box_cols]
sagittal_box_df.to_csv('sagittal_box_df.csv', index=False)

sagittal_df = pd.read_csv('sagittal_df.csv')                                                        


1it [00:00,  1.93it/s]
100%|██████████| 1/1 [00:00<00:00, 52.50it/s]


In [15]:
# --- FINAL STEP: CREATE THE CROPPED IMAGE DATASET ---

import pandas as pd
import cv2
import os
import numpy as np
from tqdm import tqdm

print("Starting the final step: Cropping images to create the dataset...")

# 1. Load the data with the final bounding box coordinates
df_to_crop = pd.read_csv('axial_with_box_df.csv')
print(f"Loaded 'axial_with_box_df.csv'. Found {len(df_to_crop)} images to process.")

# 2. Create the output directory for your new dataset
output_dir = '/kaggle/working/axial_cropped_dataset/'
os.makedirs(output_dir, exist_ok=True)
print(f"Output directory created at: {output_dir}")

# 3. Define cropping parameters (with padding, just like the 2nd place solution's models would use)
CROP_X_RATIO = 1.0
CROP_Y_RATIO = 2.0

# 4. Loop through each image, crop it, and save it
cropped_image_paths = []
for index, row in tqdm(df_to_crop.iterrows(), total=df_to_crop.shape[0], desc="Cropping and Saving Images"):
    image_path = row['path']
    image = cv2.imread(image_path)
    
    if image is None:
        cropped_image_paths.append(None)
        continue
    
    x_min, y_min, x_max, y_max = row['x_min'], row['y_min'], row['x_max'], row['y_max']
    
    width = x_max - x_min
    height = y_max - y_min
    x_pad = (width / 2) * CROP_X_RATIO
    y_pad = (height / 2) * CROP_Y_RATIO
    
    new_x_min = int(np.maximum(0, x_min - x_pad))
    new_y_min = int(np.maximum(0, y_min - y_pad))
    new_x_max = int(np.minimum(image.shape[1], x_max + x_pad))
    new_y_max = int(np.minimum(image.shape[0], y_max + y_pad))
    
    cropped_image = image[new_y_min:new_y_max, new_x_min:new_x_max]
    
    base_filename = os.path.basename(image_path)
    output_path = os.path.join(output_dir, base_filename)
    cv2.imwrite(output_path, cropped_image)
    
    cropped_image_paths.append(output_path)

# 5. Create the final metadata CSV for your new dataset
print("Finalizing metadata file...")
df_to_crop['cropped_image_path'] = cropped_image_paths
df_to_crop.dropna(subset=['cropped_image_path'], inplace=True)

final_csv_path = '/kaggle/working/axial_final_dataset_metadata.csv'
df_to_crop.to_csv(final_csv_path, index=False)

print("\n--- WORKFLOW COMPLETE! ---")
print(f"Successfully created a dataset with {len(df_to_crop)} cropped images.")
print(f"==> Image Directory: {output_dir}")
print(f"==> Metadata CSV File: {final_csv_path}")

Starting the final step: Cropping images to create the dataset...
Loaded 'axial_with_box_df.csv'. Found 5 images to process.
Output directory created at: /kaggle/working/axial_cropped_dataset/


Cropping and Saving Images: 100%|██████████| 5/5 [00:00<00:00, 390.43it/s]

Finalizing metadata file...

--- WORKFLOW COMPLETE! ---
Successfully created a dataset with 5 cropped images.
==> Image Directory: /kaggle/working/axial_cropped_dataset/
==> Metadata CSV File: /kaggle/working/axial_final_dataset_metadata.csv
